In [1]:
import os
import DALI as dali_code
dali_data_path = '/Users/bensa/MLDS/Signal Processing for AI/Lyric Sync/DALI_v2.0/annot_tismir'
dali_data = dali_code.get_the_DALI_dataset(dali_data_path, skip=[], keep=[])

In [2]:
entry = dali_data['0a0a723686924d228daef2a2f692d437']
entry.info

{'dataset_version': 0.0,
 'audio': {'url': 'le6uFnLwqYU',
  'path': 'DALI_v2.0/audio/0a0a723686924d228daef2a2f692d437.mp3',
  'working': False},
 'ground-truth': False,
 'scores': {'NCC': 0.9386545655498632,
  'manual': 0.0,
  'pitch': {'Overall Accuracy': 0.3781733186953516,
   'Raw Pitch Accuracy': 0.4444444444444444,
   'Raw Chroma Accuracy': 0.48391812865497075,
   'Voicing Recall': 0.40767937717170927,
   'Voicing False Alarm': 0.39168782106115607}},
 'artist': 'Bots',
 'title': 'Zeven Dagen Lang',
 'id': '0a0a723686924d228daef2a2f692d437',
 'metadata': {'album': 'Voor God En Vaderland',
  'release_date': '1976',
  'language': 'dutch'}}

### Get Audio
Go over all songs, and get the audio of each song.

In [ ]:
import pandas as pd
from utils.download_audio import download_and_convert_to_mp3
from tqdm import tqdm

# Initialize or load the progress dataframe
progress_file = 'download_progress.csv'
if os.path.exists(progress_file):
    progress_df = pd.read_csv(progress_file, index_col=0)
else:
    progress_df = pd.DataFrame(columns=['track_id', 'status'])

destination_folder = 'DALI/audio'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Check which tracks have already been downloaded
for track_id, track_entry in dali_data.items():
    track_path = os.path.join(destination_folder, f"{track_id}.mp3")
    if os.path.exists(track_path):
        progress_df = progress_df.append({'track_id': track_id, 'status': 'Succeeded'}, ignore_index=True)
    # else:
    #     progress_df = progress_df.append({'track_id': track_id, 'status': 'Pending'}, ignore_index=True)

# Remove duplicates if any
progress_df.drop_duplicates(subset='track_id', keep='last', inplace=True)

# Save the progress dataframe
progress_df.to_csv(progress_file)

# Filter out the tracks that need to be downloaded
tracks_to_download = progress_df[~progress_df['status'].isin(['Succeeded', 'Failed'])]

#.iloc[::-1]
for index, row in tqdm(tracks_to_download.iterrows(), total=tracks_to_download.shape[0], desc="Downloading tracks"):
    track_id = row['track_id']
    track_entry = dali_data[track_id]
    
    if download_and_convert_to_mp3(track_entry.info['audio']['url'], destination_folder, f"{track_id}.mp3"):
        progress_df.loc[progress_df['track_id'] == track_id, 'status'] = 'Succeeded'
    else:
        progress_df.loc[progress_df['track_id'] == track_id, 'status'] = 'Failed'

    # Save the progress after each attempt
    progress_df.to_csv(progress_file)

### 2. Separate vocals and accompaniment from music

In [ ]:
from spleeter.separator import Separator
import os
from tqdm import tqdm

base_dir = 'DALI/audio'
output_folder = 'DALI/vocals'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def separate_vocals(input_path, output_path):
    separator = Separator('spleeter:2stems')
    separator.separate_to_file(input_path, destination=output_path)

for track_id in tqdm(os.listdir(base_dir), desc="Processing MP3 files"):
    track_id = track_id[:-4]
    input_file = os.path.join(base_dir, f"{track_id}.mp3")
    output_file = os.path.join(output_folder, f"{track_id}_vocals.mp3")
    
    if os.path.exists(output_file):
        print(f"Skipping {track_id} as it has already been processed")
        continue
    
    try:
        print(f"Processing {track_id}")
        separate_vocals(input_file, output_folder)
        # the output file is saved as {track_id}/vocals.wav, move it to {track_id}_vocals.wav and delete the folder with the {track_id}/accompaniment.wav
        os.rename(os.path.join(output_folder, f"{track_id}/vocals.wav"), output_file)
        os.remove(os.path.join(output_folder, f"{track_id}/accompaniment.wav"))
        os.rmdir(os.path.join(output_folder, f"{track_id}"))
    except Exception as e:
        print(f"Error processing {track_id}: {e}")